## Import librairies

In [1]:
!pip install plotly
!pip install mlxtend
!pip install google
!pip install yfinance
from googlesearch import search
from yfinance import *

import pandas as pd
import numpy as np
import yfinance as yf

import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

  Using cached plotly-5.4.0-py2.py3-none-any.whl (25.3 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Using cached mlxtend-0.19.0-py2.py3-none-any.whl (1.3 MB)
  Using cached google-3.0.0-py2.py3-none-any.whl (45 kB)
  Using cached yfinance-0.1.67-py2.py3-none-any.whl (25 kB)
  Using cached lxml-4.6.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)
Processing /home/jovyan/.cache/pip/wheels/21/c9/66/b41c847de65c7985db52ec21d59996841598b8b0e93f2b9500/multitasking-0.0.10-py3-none-any.whl


In [ ]:
src.head()

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
139,/organization/21vianet,21viaNet,http://www.ch.21vianet.com,Software|Web Hosting,396000000,ipo,CHN,22,Beijing,Beijing,5,2009-10-16,2008-02-13,2014-12-04
140,/organization/22nd-century-group,22nd Century Group,http://www.xxiicentury.com,Biotechnology,15034750,ipo,USA,NY,NY - Other,Clarence,6,1998-01-01,2010-12-28,2014-09-18
201,/organization/2u,2U Inc.,http://2u.com,EdTech|Education|Software,95875000,ipo,USA,MD,MD - Other,Hyattsville,6,2008-01-01,2009-03-25,2013-10-21
270,/organization/3d-systems,3D Systems,http://www.3dsystems.com,Software,169500000,ipo,USA,SC,SC - Other,Rock Hill,3,1986-01-01,2014-01-15,2014-10-13
436,/organization/58-com,58.com,http://www.58.com,Curated Web|Internet|Software,876780000,ipo,CHN,19,CHN - Other,Chaoyang,6,2005-01-01,2006-02-01,2014-06-27


## Get companies name from the file companies and filter on companies which qhoted on stock market

In [2]:
src=pd.read_csv(r'src/companies.csv', encoding='latin1')

In [3]:
src.shape

(66368, 14)

**Filtering IPO companies( IPO means that a companie lists its shares on a stock exchange)**

In [4]:
mask= src["status"] =="ipo"
src= src[mask]
dfi= pd.DataFrame()
dfi["name"] = src["name"]

In [5]:
dfi.shape

(1547, 1)

In [6]:
dfi=dfi.head(1600)
dfi.head(10)

,name
139,21viaNet
140,22nd Century Group
201,2U Inc.
270,3D Systems
436,58.com
442,5BARz International
468,6D Global Technologies
543,8x8 Inc
639,A10 Networks
640,A123 Systems


**Function that launch a google search to collect the ticker matching each companie name
A ticker is like a key that is used to reference a unique companie
Ex for Apple the ticker is AAPL**

In [ ]:
def name_convert(self):

    searchval = 'yahoo finance '+self
    link = []
    #limits to the first link
    for url in search(searchval, tld='com', lang='en', stop=1):
        link.append(url)

    link = str(link[0])
    link=link.split("/")
    if link[-1]=='':
        ticker=link[-2]
    else:
        x=link[-1].split('=')
        ticker=x[-1]

    return(ticker)

**Collecting all Tickers for companies, the result is saved in a separated file in the src folder**

In [ ]:
dfi["name_ticker"] = dfi["name"].apply(name_convert)
dfi["lenticker"]=dfi["name_ticker"].apply(lambda x: len(x))
dfi = dfi[dfi["lenticker"]<10]
dfi.to_csv("src/save_ticker.csv")

**Getting companie decription and stocks history(last 3month), the result is saved in a separated file in the src folder**

In [8]:
dfi_tick =pd.read_csv("src/save_ticker.csv")
dfi_tick =dfi_tick.drop(['Unnamed: 0'], axis=1)

temp= pd.DataFrame()
df_hist = pd.DataFrame()
i=0
for tk in dfi_tick["name_ticker"]:    
    #get stock price(only Closed value) evolution history (last 3month)
    print("Row number",i,"ticker",tk) 
    stock= yf.Ticker(tk)
    temp= stock.history(period="3mo")
    temp=temp.reset_index()
    temp= temp[["Date","Close"]]
    temp["tick"]=tk
    mask=  dfi_tick["name_ticker"] ==tk
    temp["companie"]=dfi_tick[mask].iloc[0,0]
    
    #get companie description
    msft = yf.Ticker(tk).info  
    x = msft.get('longBusinessSummary')
    temp["description"]=x
    df_hist= df_hist.append(temp,ignore_index=True)
    
    #Save each 100 companies infos collected 
    if i % 100 == 0:
        df_hist.to_csv('src/save_yfinance22112021.csv', mode='a', header=False)
        df_hist = pd.DataFrame()
    i +=1

    dff =pd.read_csv("src/save_yfinance22112021.csv",header=None, 
                  names=["Date", "Stock_value","Companie_Ticker", "Companie_name","Companie_Desc"])
    
     #formating the Stock price to avoid infinity precision digits after coma
    dff["Stock_value"]=dff["Stock_value"].apply(lambda x: round(float(x), 2))
    #Save result to csv file
    dff.to_csv('src/save_yfinance_clean22112021.csv')

Row number 0 ticker VNET
Row number 1 ticker XXII
Row number 2 ticker TWOU
Row number 3 ticker DDD
Row number 4 ticker WUBA
- WUBA: No data found, symbol may be delisted
Row number 5 ticker SIXD
Row number 6 ticker EGHT
Row number 7 ticker ATEN
Row number 8 ticker profile
- PROFILE: No data found, symbol may be delisted
Row number 9 ticker aacay
Row number 10 ticker ASTM
- ASTM: No data found for this date range, symbol may be delisted
Row number 11 ticker ATTBF
Row number 12 ticker ABB
Row number 13 ticker ABT
Row number 14 ticker ABLX
- ABLX: No data found for this date range, symbol may be delisted
Row number 15 ticker ABHDD
- ABHDD: No data found for this date range, symbol may be delisted
Row number 16 ticker ABZA.L
- ABZA.L: No data found for this date range, symbol may be delisted
Row number 17 ticker actg
Row number 18 ticker ACAD
Row number 19 ticker ACNV
Row number 20 ticker AXDX
Row number 21 ticker xlrn
Row number 22 ticker NGM
Row number 23 ticker ACCO
Row number 24 ticker